In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
from collections import defaultdict

folders = ['pre', 'non', 'type2']

files = defaultdict(list)

for folder in folders:
    folder_path = f'data/{folder}'
    for csv in os.listdir(folder_path):
        path = os.path.join(folder_path, csv)
        files[folder].append(path)

In [3]:
print(files)

defaultdict(<class 'list'>, {'pre': ['data/pre/CGMacros-009.csv', 'data/pre/CGMacros-008.csv', 'data/pre/CGMacros-020.csv', 'data/pre/CGMacros-022.csv', 'data/pre/CGMacros-023.csv', 'data/pre/CGMacros-032.csv', 'data/pre/CGMacros-026.csv', 'data/pre/CGMacros-043.csv', 'data/pre/CGMacros-041.csv', 'data/pre/CGMacros-044.csv', 'data/pre/CGMacros-045.csv', 'data/pre/CGMacros-029.csv', 'data/pre/CGMacros-016.csv', 'data/pre/CGMacros-013.csv', 'data/pre/CGMacros-007.csv', 'data/pre/CGMacros-011.csv', 'data/pre/CGMacros-010.csv'], 'non': ['data/non/CGMacros-021.csv', 'data/non/CGMacros-034.csv', 'data/non/CGMacros-027.csv', 'data/non/CGMacros-033.csv', 'data/non/CGMacros-018.csv', 'data/non/CGMacros-031.csv', 'data/non/CGMacros-019.csv', 'data/non/CGMacros-048.csv', 'data/non/CGMacros-001.csv', 'data/non/CGMacros-015.csv', 'data/non/CGMacros-017.csv', 'data/non/CGMacros-002.csv', 'data/non/CGMacros-006.csv', 'data/non/CGMacros-004.csv'], 'type2': ['data/type2/CGMacros-035.csv', 'data/type2/C

In [4]:
import os
import pandas as pd

def get_averages(file_path):
    df = pd.read_csv(file_path)

    file_name = os.path.basename(file_path)
    file_name_no_ext = os.path.splitext(file_name)[0]  
    participant_id = int(file_name_no_ext.split('-')[-1])

    if file_path == 'data/type2/CGMacros-030.csv':
        df = df[['Timestamp', 'Dexcom GL']]
        df = df.rename(columns={'Timestamp': 'time', 'Dexcom GL': 'gl'})
    else:
        df = df[['Timestamp', 'Libre GL']]
        df = df.rename(columns={'Timestamp': 'time', 'Libre GL': 'gl'})

    df['time'] = pd.to_datetime(df['time'])
    df['date'] = df['time'].dt.date

    daily_avg = df.groupby('date')['gl'].mean().reset_index()
    daily_avg.sort_values(by='date', inplace=True)
    daily_avg.reset_index(drop=True, inplace=True)

    averages = daily_avg['gl'].values
    num_days = len(averages)
    day_columns = [f'day{i+1}' for i in range(num_days)]
    
    result_df = pd.DataFrame([averages], columns=day_columns, index=[participant_id])
    result_df.index.name = 'participant_id'

    return result_df

In [150]:
get_averages('data/type2/CGMacros-003.csv').head()

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,day11
participant_id,,,,,,,,,,,
3,99.100557,107.776806,108.439439,117.839861,103.122222,104.803194,113.468056,106.17125,107.141528,108.198472,100.569571


In [5]:
averages = []

for folder in files:
    for file_path in files[folder]:
        df = get_averages(file_path)
        averages.append(df)

average_by_day = pd.concat(averages)
average_by_day = average_by_day.sort_index()
average_by_day.head()

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,day11,day12,day13,day14,day15,day16,day17,day18,day19,day20
participant_id,,,,,,,,,,,,,,,,,,,,
1,84.729630,88.055556,88.923611,90.785417,89.125000,89.095139,91.863194,87.656944,88.314583,83.650000,90.493750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53.194737,72.771111,77.848889,68.761528,79.847083,83.618333,89.764028,79.784722,81.892500,80.895417,80.399444,77.965,78.080878,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99.100557,107.776806,108.439439,117.839861,103.122222,104.803194,113.468056,106.171250,107.141528,108.198472,100.569571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,91.611111,91.910417,93.678472,98.241667,87.280556,86.262500,89.511806,85.127869,90.540972,94.367361,88.140725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,94.801567,102.064583,109.162500,118.669444,105.314583,117.194444,121.789583,105.941667,117.634028,112.718750,115.320163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
days = []
for i in range(1, 11):
    days.append(f'day{i}')

average_by_day = average_by_day[days]
average_by_day.head()

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10
participant_id,,,,,,,,,,
1,84.729630,88.055556,88.923611,90.785417,89.125000,89.095139,91.863194,87.656944,88.314583,83.650000
2,53.194737,72.771111,77.848889,68.761528,79.847083,83.618333,89.764028,79.784722,81.892500,80.895417
3,99.100557,107.776806,108.439439,117.839861,103.122222,104.803194,113.468056,106.171250,107.141528,108.198472
4,91.611111,91.910417,93.678472,98.241667,87.280556,86.262500,89.511806,85.127869,90.540972,94.367361
5,94.801567,102.064583,109.162500,118.669444,105.314583,117.194444,121.789583,105.941667,117.634028,112.718750


In [7]:
average_by_day.to_csv('time_series.csv')